In [46]:
import requests
import json

import pandas as pd
import numpy as np
import aqi

from pandas import concat
import pickle

import tensorflow as tf
from tensorflow import keras

from datetime import datetime, time, timedelta,date
import time as t

In [67]:
import sklearn as sk

In [68]:
sk.__version__

'1.0.1'

In [2]:
#Modify these functions as we dont need just single row with lag features and no y value.

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def data_split(data,n_hours,n_features,reshape):
    values = data.values
    n_obs = n_hours * n_features
    QA_X, QA_y = values[:, :n_obs], values[:, -n_features]
    if reshape=='yes':
        QA_X  = QA_X.reshape((QA_X.shape[0], n_hours, n_features))
    
    return QA_X 

In [3]:
def AQI_cal(value,pollutant):
    
    if value <0:
        value=0
    
    if pollutant=='CO':
        if value <=50:
            maqi=aqi.to_iaqi(aqi.POLLUTANT_CO_8H, value, algo=aqi.ALGO_EPA)
        else :
            maqi=500

    elif pollutant=='OZONE':
        maqi= aqi.to_iaqi(aqi.POLLUTANT_O3_8H, value, algo=aqi.ALGO_EPA)
        
    elif pollutant=='PM25':
        if value <=500:
            maqi= aqi.to_iaqi(aqi.POLLUTANT_PM25, value, algo=aqi.ALGO_EPA)
        else:
            maqi=500
        
    elif pollutant=='PM10':
        if value <=600:
            maqi= aqi.to_iaqi(aqi.POLLUTANT_PM10, value, algo=aqi.ALGO_EPA)
        else:
            maqi=600
        
    return maqi

In [4]:
def AQI_level(value):
    if value <= 50:
        return "Good"
    elif value <= 100:
        return "Moderate"
    elif value <= 150:
        return "UnHealthy for Sensitive Groups"
    elif value <= 200:
        return "UnHealthy"
    elif value <= 300:
        return "Very UnHealthy"
    elif value > 300:
        return "Hazardous"
    else:
        return np.NaN

In [5]:
#prepare data for models
#Load scaler
scaler=pickle.load(open('scaler.pkl', 'rb'))

In [6]:
Weather_API_KEY='7872c767e8eaf7d60f3ff0970f218a4e'

In [7]:
#ret=requests.get('http://api.openweathermap.org/data/2.5/weather?q=San Jose,us&units=imperial&APPID=7872c767e8eaf7d60f3ff0970f218a4e')

#ret=requests.get('https://api.openweathermap.org/data/2.5/onecall?lat=33.44&lon=-94.04&units=imperial&exclude=hourly,daily,minutely,alerts&APPID=7872c767e8eaf7d60f3ff0970f218a4e')

In [8]:
def get_today_unix(dt_in): #function to get unix time for local time
    unix_dt=int(t.mktime(dt_in.timetuple()))
    return unix_dt    

In [9]:
#import datetime
def change_date(dt):
    return(datetime.fromtimestamp(int(dt)).strftime('%Y-%m-%d %H:%M:%S'))

In [10]:
def weather_api_call(lat,long,dt,key):
    return requests.get(f'https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={long}&units=imperial&dt={dt}&appid={key}')

In [11]:
lat=33.298600#33.44
long=-115.956100#-94.04

In [12]:
date.today()

datetime.date(2021, 11, 17)

In [13]:
#Barometric_Pressure 	Dewpoint 	Relative_Humidity 	Temperature 	Wind_Speed

In [14]:
#Code to Get previous 5 hours of weather data
if time(17,0) <= datetime.now().time() < time(21,0):
    print("Hello There")
    #call twice by using cuurent time and time of previous date.
    
    dt1=get_today_unix(datetime.combine(date.today(), datetime.min.time())) #previous utc day
    
    hist_resp1=weather_api_call(lat,long,dt1,Weather_API_KEY)
    main_list=[]
    for i in hist_resp1.json()['hourly']:
        main_list.append([i['dt'],i['pressure'],i['dew_point'],i['humidity'],i['temp'],i['wind_speed']])

    dt2=get_today_unix(date.today()) #new UTC day
    
    #hist_resp2=requests.get(f'https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=33.44&lon=-94.04&units=imperial&dt={dt2}&appid={Weather_API_KEY}') 
    hist_resp2=weather_api_call(lat,long,dt2,Weather_API_KEY)
    for i in hist_resp2.json()['hourly']:
        main_list.append([i['dt'],i['pressure'],i['dew_point'],i['humidity'],i['temp'],i['wind_speed']])
    
    df=pd.DataFrame(main_list,columns=['date','Barometric_Pressure','Dewpoint','Relative_Humidity','Temperature','Wind_Speed'])

    df['date']=df.date.apply(change_date)
    
elif time(21,0) <= datetime.now().time() < time(23,59):
    print("Hello")
    
    dt_in=datetime.today().replace(hour=17, minute=0, second=0, microsecond=0)
    dt=get_today_unix(dt_in)
    print(dt)      
    
    hist_resp=weather_api_call(lat,long,dt,Weather_API_KEY)
    main_list=[]
    for i in hist_resp.json()['hourly']:
        main_list.append([i['dt'],i['pressure'],i['dew_point'],i['humidity'],i['temp'],i['wind_speed']])

    df=pd.DataFrame(main_list,columns=['date','Barometric_Pressure','Dewpoint','Relative_Humidity','Temperature','Wind_Speed'])

    df['date']=df.date.apply(change_date)

else:
    print("else")
    
    dt=get_today_unix(date.today())
    print(dt)
    
    hist_resp=weather_api_call(lat,long,dt,Weather_API_KEY)
    
    main_list=[]
    for i in hist_resp.json()['hourly']:
        main_list.append([i['dt'],i['pressure'],i['dew_point'],i['humidity'],i['temp'],i['wind_speed']])

    df=pd.DataFrame(main_list,columns=['date','Barometric_Pressure','Dewpoint','Relative_Humidity','Temperature','Wind_Speed'])

    df['date']=df.date.apply(change_date)
    
    
    
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['date'],inplace=True)
df.tail(5)

else
1637136000


,date,Barometric_Pressure,Dewpoint,Relative_Humidity,Temperature,Wind_Speed
15,2021-11-17 07:00:00,1014,26.58,28,58.89,1.79
16,2021-11-17 08:00:00,1014,35.02,25,72.86,1.07
17,2021-11-17 09:00:00,1015,55.99,55,73.11,0.83
18,2021-11-17 10:00:00,1015,53.11,43,77.32,2.46
19,2021-11-17 11:00:00,1015,57.18,46,79.79,2.77


In [15]:
gt=requests.get(f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={long}&appid={Weather_API_KEY}')

#gt=requests.get(f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={long}&start={1636228800}&appid={Weather_API_KEY}')
gt.json()

{'coord': {'lon': -115.9561, 'lat': 33.2986},
 'list': [{'main': {'aqi': 4},
   'components': {'co': 280.38,
    'no': 0.95,
    'no2': 4.93,
    'o3': 86.55,
    'so2': 0.22,
    'pm2_5': 28.22,
    'pm10': 60.37,
    'nh3': 19},
   'dt': 1637179200}]}

In [16]:
#Functions to change units of wind speed to knots
def wind_speed_change(wind_in):
    return (wind_in*0.868976)

In [17]:
df['Wind_Speed']=df.Wind_Speed.apply(wind_speed_change)

In [18]:
df.tail(5)

,date,Barometric_Pressure,Dewpoint,Relative_Humidity,Temperature,Wind_Speed
15,2021-11-17 07:00:00,1014,26.58,28,58.89,1.555467
16,2021-11-17 08:00:00,1014,35.02,25,72.86,0.929804
17,2021-11-17 09:00:00,1015,55.99,55,73.11,0.721250
18,2021-11-17 10:00:00,1015,53.11,43,77.32,2.137681
19,2021-11-17 11:00:00,1015,57.18,46,79.79,2.407064


### Air Quality Data Download

In [19]:
hours = 5
current_date_and_time = datetime.today().replace( minute=0, second=0, microsecond=0) #datetime.now()

hours_added = timedelta(hours = hours)

start_hour = current_date_and_time - hours_added

start_hour

datetime.datetime(2021, 11, 17, 7, 0)

In [20]:
start_time=get_today_unix(start_hour)

In [21]:
end_hour=datetime.today().replace( minute=0, second=0, microsecond=0)
end_time=get_today_unix(end_hour)

In [22]:
req2=requests.get(f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={long}&start={start_time}&end={end_time}&appid={Weather_API_KEY}')

In [23]:
req2

<Response [200]>

In [24]:
air_list=[]
for i in (req2.json()['list']):
    print(i['components'],i['dt'])
    air_list.append([i['dt'],i['components']['co'],i['components']['no2'],i['components']['o3'],i['components']['pm10'],i['components']['pm2_5'],i['components']['so2']])

{'co': 277.04, 'no': 0.01, 'no2': 6.34, 'o3': 46.49, 'so2': 0.22, 'pm2_5': 16.22, 'pm10': 37.72, 'nh3': 11.4} 1637161200
{'co': 277.04, 'no': 0.37, 'no2': 6.94, 'o3': 45.78, 'so2': 0.2, 'pm2_5': 17.14, 'pm10': 41.44, 'nh3': 13.05} 1637164800
{'co': 280.38, 'no': 1.24, 'no2': 6.77, 'o3': 48.64, 'so2': 0.2, 'pm2_5': 18.73, 'pm10': 47.27, 'nh3': 16.97} 1637168400
{'co': 280.38, 'no': 1.72, 'no2': 6.51, 'o3': 55.08, 'so2': 0.19, 'pm2_5': 21.69, 'pm10': 53.97, 'nh3': 20.52} 1637172000
{'co': 283.72, 'no': 1.52, 'no2': 6, 'o3': 66.52, 'so2': 0.18, 'pm2_5': 24.98, 'pm10': 59.09, 'nh3': 21.28} 1637175600
{'co': 280.38, 'no': 0.95, 'no2': 4.93, 'o3': 86.55, 'so2': 0.22, 'pm2_5': 28.22, 'pm10': 60.37, 'nh3': 19} 1637179200


In [25]:
df_air=pd.DataFrame(air_list,columns=['date','CO','NO2','OZONE','PM10','PM25','SO2'])

df_air['date']=df_air.date.apply(change_date)

In [26]:
df_air['date'] = pd.to_datetime(df_air['date'])
df_air.sort_values(by=['date'],inplace=True)

In [27]:
#change unit to ppm
def ppm_unit(poll_in):
    return (0.001*poll_in)

In [28]:
df_air['CO']=df_air['CO'].apply(ppm_unit)
df_air['NO2']=df_air['NO2'].apply(ppm_unit)
df_air['OZONE']=df_air['OZONE'].apply(ppm_unit)
df_air['SO2']=df_air['SO2'].apply(ppm_unit)

In [29]:
df_air.tail(5)

,date,CO,NO2,OZONE,PM10,PM25,SO2
1,2021-11-17 08:00:00,0.27704,0.00694,0.04578,41.44,17.14,0.00020
2,2021-11-17 09:00:00,0.28038,0.00677,0.04864,47.27,18.73,0.00020
3,2021-11-17 10:00:00,0.28038,0.00651,0.05508,53.97,21.69,0.00019
4,2021-11-17 11:00:00,0.28372,0.00600,0.06652,59.09,24.98,0.00018
5,2021-11-17 12:00:00,0.28038,0.00493,0.08655,60.37,28.22,0.00022


In [30]:
df.tail(5)

,date,Barometric_Pressure,Dewpoint,Relative_Humidity,Temperature,Wind_Speed
15,2021-11-17 07:00:00,1014,26.58,28,58.89,1.555467
16,2021-11-17 08:00:00,1014,35.02,25,72.86,0.929804
17,2021-11-17 09:00:00,1015,55.99,55,73.11,0.721250
18,2021-11-17 10:00:00,1015,53.11,43,77.32,2.137681
19,2021-11-17 11:00:00,1015,57.18,46,79.79,2.407064


In [31]:
new_df=pd.concat([df_air.drop(columns='date').tail(5).reset_index(),df.drop(columns='date').tail(5).reset_index()],axis=1)

In [32]:
new_df.drop(columns='index',inplace=True)

In [33]:
#Dataset to hold final results
df_model_results = pd.DataFrame(columns = ['CO_Predicted','AQI_CO_Predicted',
                                           'Ozone_Predicted','AQI_Ozone_Predicted',
                                           'PM25_Predicted','AQI_PM25_Predicted',
                                           'PM10_Predicted','AQI_PM10_Predicted',
                                           'Final_AQI_Predicted'])

In [34]:
#check units for air quality

#load all the models

#Now scale this data with saved scaler.
#place check for Nan after scaling.

#Change to supervised format
#call all the models
#Invert scaling for each
#Save results in dataframe
#Calculate individial AQI
#Calculate final AQI and individual parameters 

In [35]:
model_CO = tf.keras.models.load_model('best_model_CO.h5')

model_Ozone = tf.keras.models.load_model('best_model_Ozone.h5')

Pkl_Filename = "best_model_PM25.pkl"  

# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    model_PM25 = pickle.load(file)
    
Pkl_Filename = "best_model_PM10.pkl"  

# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    model_PM10 = pickle.load(file)

In [36]:
#Perform Robust scaling, change it with standard scaler , if we dont get good results
columns_transform=list(new_df.columns) #numerical columns to be transformed.

values=new_df.values

scaled=scaler.fit_transform(values)

df_transformed=pd.DataFrame(scaled,columns=new_df.columns)

df_transformed

,CO,NO2,OZONE,PM10,PM25,SO2,Barometric_Pressure,Dewpoint,Relative_Humidity,Temperature,Wind_Speed
0,-1.581139,0.981390,-0.992664,-1.532103,-1.238669,0.150756,-1.224745,-1.528616,-1.009839,-1.865314,0.007948
1,0.000000,0.746409,-0.799979,-0.719202,-0.845715,0.150756,-1.224745,-0.849446,-1.275586,0.064369,-0.945862
2,0.000000,0.387027,-0.366101,0.215008,-0.114179,-0.603023,0.816497,0.838019,1.381884,0.098901,-1.263799
3,1.581139,-0.317915,0.404638,0.928911,0.698914,-1.356801,0.816497,0.606264,0.318896,0.680431,0.895522
4,0.000000,-1.796911,1.754106,1.107387,1.499649,1.658312,0.816497,0.933779,0.584643,1.021613,1.306191


In [37]:
#Add Dummy Row

series_obj = pd.Series( [0,0,0,0,0,0,0,0,0,0,0], 
                        index=df_transformed.columns )


# Add a series as a row to the dataframe  
mod_df = df_transformed.append(  series_obj,
                        ignore_index=True)

In [38]:
#CO Prediction
# specify the number of lag hours
n_hours = 5
n_features = 9 #number of features in dataframe to be used

# frame as supervised learning with previous 5 hours data to predict upcoming hour data.
scaled_CO=mod_df.iloc[:,[0,1,2,3,5,6,8,9,10]].values
reframed_CO = series_to_supervised(scaled_CO, n_hours, 1)
print(reframed_CO.shape)

CO_X=data_split(reframed_CO,n_hours,n_features,'yes')

yhat=model_CO.predict(CO_X)
yhat = np.repeat(yhat, 11, axis=-1)
inv_yhat = scaler.inverse_transform(yhat)[:,0]

df_model_results['CO_Predicted']=inv_yhat

df_model_results['AQI_CO_Predicted']=df_model_results.CO_Predicted.apply(AQI_cal,pollutant='CO')

(1, 54)


In [39]:
#Ozone Prediction
# specify the number of lag hours
n_hours = 5
n_features = 6 #number of features in dataframe to be used

# frame as supervised learning with previous 5 hours data to predict upcoming hour data.
scaled=mod_df.iloc[:,[2,0,1,8,9,10]].values
reframed = series_to_supervised(scaled, n_hours, 1)
print(reframed.shape)

Ozone_X=data_split(reframed,n_hours,n_features,'yes')

yhat=model_Ozone.predict(Ozone_X)
yhat = np.repeat(yhat, 11, axis=-1)
inv_yhat = scaler.inverse_transform(yhat)[:,2]

df_model_results['Ozone_Predicted']=inv_yhat

df_model_results['AQI_Ozone_Predicted']=df_model_results.Ozone_Predicted.apply(AQI_cal,pollutant='OZONE')

(1, 36)


In [40]:
#PM2.5 Prediction
# specify the number of lag hours
n_hours = 5
n_features = 7 #number of features in dataframe to be used

# frame as supervised learning with previous 5 hours data to predict upcoming hour data.
scaled=mod_df.iloc[:,[4,3,6,7,8,9,10]].values
reframed = series_to_supervised(scaled, n_hours, 1)
print(reframed.shape)

PM25_X=data_split(reframed,n_hours,n_features,'no')

yhat=model_PM25.predict(PM25_X)
yhat = np.repeat(yhat.reshape(-1,1), 11, axis=-1)
inv_yhat = scaler.inverse_transform(yhat)[:,4]

df_model_results['PM25_Predicted']=inv_yhat

df_model_results['AQI_PM25_Predicted']=df_model_results.PM25_Predicted.apply(AQI_cal,pollutant='PM25')

(1, 42)


In [41]:
#PM10 Prediction
# specify the number of lag hours
n_hours = 5
n_features = 7 #number of features in dataframe to be used

# frame as supervised learning with previous 5 hours data to predict upcoming hour data.
scaled=mod_df.iloc[:,[3,0,1,2,4,5,10]].values
reframed = series_to_supervised(scaled, n_hours, 1)
print(reframed.shape)

PM10_X=data_split(reframed,n_hours,n_features,'no')

yhat=model_PM10.predict(PM10_X)
yhat = np.repeat(yhat.reshape(-1,1), 11, axis=-1)
inv_yhat = scaler.inverse_transform(yhat)[:,3]

df_model_results['PM10_Predicted']=inv_yhat

df_model_results['AQI_PM10_Predicted']=df_model_results.PM10_Predicted.apply(AQI_cal,pollutant='PM10')

(1, 42)


In [42]:
df_model_results['Final_AQI_Predicted'] = (df_model_results[["AQI_CO_Predicted", "AQI_Ozone_Predicted",
                                                                 "AQI_PM25_Predicted", "AQI_PM10_Predicted"]].max(axis = 1))

df_model_results['Predicted_AQI_LEVEL'] = df_model_results["Final_AQI_Predicted"].apply(AQI_level)

In [43]:
df_model_results

,CO_Predicted,AQI_CO_Predicted,Ozone_Predicted,AQI_Ozone_Predicted,PM25_Predicted,AQI_PM25_Predicted,PM10_Predicted,AQI_PM10_Predicted,Final_AQI_Predicted,Predicted_AQI_LEVEL
0,0.279762,2,0.082461,116,26.261755,81,59.156654,53,116.0,UnHealthy for Sensitive Groups


In [44]:
CO=((round(df_model_results.CO_Predicted,2)).values)[0]
Ozone=((round(df_model_results.Ozone_Predicted,4)).values)[0]
PM25=((round(df_model_results.PM25_Predicted,2)).values)[0]
PM10=((round(df_model_results.PM10_Predicted,2)).values)[0]
AQI_level=df_model_results.Predicted_AQI_LEVEL.values[0]
AQI_value=df_model_results.Final_AQI_Predicted.values[0]

In [45]:
print(CO)

0.28
